<a href="https://colab.research.google.com/github/vggd18/pyspark-etl-acidentes-recife/blob/main/etl_acidentes_recife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Data Lakehouse: Acidentes de Trânsito do Recife

**Objetivo:** Construir um pipeline de dados completo (ETL) utilizando PySpark e Delta Lake para processar dados abertos de acidentes de trânsito da cidade do Recife. O projeto segue a arquitetura Medalhão (Bronze, Silver, Gold) para criar um Data Lakehouse robusto, otimizado e pronto para análises.

**Ferramentas:** PySpark, Delta Lake, Python

---

## 1. Configuração do Ambiente (Environment Setup)

Nesta seção, preparamos nosso ambiente de desenvolvimento no Google Colab, instalando as bibliotecas necessárias e configurando a sessão Spark com suporte ao Delta Lake.

### 1.1. Download dos Dados de Origem (Source Data)

A primeira etapa do pipeline é a ingestão dos dados brutos. Aqui, fazemos o download dos arquivos CSV anuais (2019-2024) diretamente do [Portal de Dados Abertos da Prefeitura do Recife](http://dados.recife.pe.gov.br/dataset/acidentes-de-transito-com-e-sem-vitimas).

In [1]:
import os
import requests

urls = {
  "2019": "http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/3531bafe-d47d-415e-b154-a881081ac76c/download/acidentes-2019.csv",
  "2020": "http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/fc1c8460-0406-4fff-b51a-e79205d1f1ab/download/acidentes_2020-novo.csv",
  "2021": "http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/2caa8f41-ccd9-4ea5-906d-f66017d6e107/download/acidentes2021.csv",
  "2022": "http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/971e0228-fa9c-4a42-b360-c842b29d2f56/download/acidentes2022.csv",
  "2023": "http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/d26b864b-0f7b-403e-b142-fd9989acaaf5/download/acidentes2023.csv",
  "2024": "http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/29afbf42-a36c-475c-8b75-761e17e67679/download/acidentes2024.csv"
}

output_dir = 'data'
os.makedirs(output_dir, exist_ok=True)

for year, url in urls.items():
  file_name = f"acidentes_{year}.csv"
  file_path =  os.path.join(output_dir, file_name)

  response = requests.get(url)

  if response.status_code == 200:
    with open(file_path, 'wb') as file:
      file.write(response.content)
    print(f"Archive {file_name} downloaded successfully.")
  else:
    print(f"Failed to download {file_name}. Status code: {response.status_code}")

print("All files downloaded successfully.")



Archive acidentes_2019.csv downloaded successfully.
Archive acidentes_2020.csv downloaded successfully.
Archive acidentes_2021.csv downloaded successfully.
Archive acidentes_2022.csv downloaded successfully.
Archive acidentes_2023.csv downloaded successfully.
Archive acidentes_2024.csv downloaded successfully.
All files downloaded successfully.


### 1.2. Inicialização da Sessão Spark (Spark Session)

Configuramos uma sessão Spark habilitada para o Delta Lake. Utilizamos a função `configure_spark_with_delta_pip` que garante a correta configuração das dependências Java (JARs), resolvendo os desafios de compatibilidade do ambiente.

In [2]:
!pip install pyspark==3.5.1 delta-spark==3.2.0 -q

In [3]:
from pyspark.sql import SparkSession
from delta import *

builder = (
  SparkSession.builder.appName("EtlAcidentesRecife")
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

print("SparkSession and Delta Lake configured successfully!")

SparkSession and Delta Lake configured successfully!


## 2. Camada Bronze: Ingestão e Armazenamento dos Dados Brutos

O objetivo da Camada Bronze é criar uma cópia fiel, histórica e imutável dos dados de origem. Nesta etapa, lemos todos os arquivos CSV, lidamos com as inconsistências de schema e salvamos os dados em uma única tabela Delta particionada.

### 2.1. Análise de Consistência do Schema (Schema Drift Analysis)

Antes de carregar todos os arquivos, é uma boa prática verificar se eles possuem a mesma estrutura. Nosso script de análise revelou um **Schema Drift** significativo: os nomes e o número de colunas mudam ao longo dos anos. Esta descoberta é crucial e justifica a necessidade de um schema unificado manual.

In [4]:
df_acidentes = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ";") \
  .csv('data/acidentes_2019.csv')

In [5]:
df_acidentes.printSchema()

root
 |-- DATA: date (nullable = true)
 |-- hora: string (nullable = true)
 |-- natureza_acidente: string (nullable = true)
 |-- situacao: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- detalhe_endereco_acidente: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- endereco_cruzamento: string (nullable = true)
 |-- numero_cruzamento: string (nullable = true)
 |-- referencia_cruzamento: string (nullable = true)
 |-- bairro_cruzamento: string (nullable = true)
 |-- num_semaforo: integer (nullable = true)
 |-- sentido_via: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- auto: integer (nullable = true)
 |-- moto: integer (nullable = true)
 |-- ciclom: integer (nullable = true)
 |-- ciclista: integer (nullable = true)
 |-- pedestre: integer (nullable = true)
 |-- onibus: integer (nullable = true)
 |-- caminhao: inte

In [6]:
df_acidentes.show(5, truncate=False)

+----------+--------+-----------------+----------+-----------+-------------------------------+------+---------------------------+-----------------------------------------------------------+-------------------------------+-----------------+-----------------------------------------------------------+-----------------+------------+-----------+------------------------+-----------------------------------------------------------------------------------+----+----+------+--------+--------+------+--------+-------+------+-------+-------------+-------------------+-----------+-----------------+---------------+------------+---------------+-----------------+--------------+------------------+-----------+----------------+------------+------------+
|DATA      |hora    |natureza_acidente|situacao  |bairro     |endereco                       |numero|detalhe_endereco_acidente  |complemento                                                |endereco_cruzamento            |numero_cruzamento|referencia_cruzamen

In [7]:
df_acidentes.describe().show()

+-------+--------+-----------------+---------+---------+-------------------+------------------+-------------------------+--------------------+-------------------+------------------+---------------------+-----------------+------------------+-----------------+--------------------+------------------+------------------+------------------+------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------+-------------------+-----------+-----------------+---------------+------------+---------------+--------------+--------------+------------------+-----------+------------+------------+------------+
|summary|    hora|natureza_acidente| situacao|   bairro|           endereco|            numero|detalhe_endereco_acidente|         complemento|endereco_cruzamento| numero_cruzamento|referencia_cruzamento|bairro_cruzamento|      num_semaforo|      sentido_via|                tipo|         descricao|          

In [8]:
import os
import csv

data_dir = 'data/'

csv_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.csv')])

if not csv_files:
  print("No CSV files found in the directory 'data/'")
else:
  base_path = os.path.join(data_dir, csv_files[0])
  base_header = spark.read.option("delimiter", ";").option("header", "true").csv(base_path).columns

  equals = True

  for f in csv_files[1:]:
    current_path = os.path.join(data_dir, f)
    current_header = spark.read.option("delimiter", ";").option("header", "true").csv(current_path).columns
    if current_header != base_header:
      equals = False
      print(f"\n!!! ALERT: The header of '{f}' is DIFFERENT! Analysis:")
      base_set = set(base_header)
      current_set = set(current_header)
      removed_columns = base_set - current_set
      if removed_columns:
          print(f"  - Missing columns in this file: {list(removed_columns)}")
      added_columns = current_set - base_set
      if added_columns:
          print(f"  - Extra columns found in this file: {list(added_columns)}")
      if len(base_header) != len(current_header):
            print(f"  - Column count diverges: {len(base_header)} in reference vs. {len(current_header)} in this file.") # Translated
      print("-" * 30)

  if equals:
      print("\nGreat news! All CSV files have the same header.")


!!! ALERT: The header of 'acidentes_2020.csv' is DIFFERENT! Analysis:
  - Missing columns in this file: ['endereco_cruzamento', 'DATA', 'numero_cruzamento', 'referencia_cruzamento']
  - Extra columns found in this file: ['data']
  - Column count diverges: 41 in reference vs. 38 in this file.
------------------------------

!!! ALERT: The header of 'acidentes_2021.csv' is DIFFERENT! Analysis:
  - Missing columns in this file: ['numero_cruzamento', 'DATA', 'endereco_cruzamento', 'descricao', 'referencia_cruzamento']
  - Extra columns found in this file: ['data']
  - Column count diverges: 41 in reference vs. 37 in this file.
------------------------------

!!! ALERT: The header of 'acidentes_2022.csv' is DIFFERENT! Analysis:
  - Missing columns in this file: ['numero_cruzamento', 'DATA', 'endereco_cruzamento', 'descricao', 'natureza_acidente', 'referencia_cruzamento']
  - Extra columns found in this file: ['natureza', 'Protocolo', 'data']
  - Column count diverges: 41 in reference vs. 3

In [9]:
df_acidentes2024 = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ";") \
  .csv('data/acidentes_2024.csv')

df_acidentes2024.describe().show()

+-------+-----------+------------+----------+-------+--------------------+-----------------+-------------------------+--------------------+-----------------+------------+-----------+--------------------+----+----+------+--------+--------+------+--------+-------+------+-------+-------------+-------------------+-----------+-----------------+-----------+------------+---------------+--------------+--------------+------------------+-----------+------------+------------+------------+
|summary|  Protocolo|    natureza|  situacao| bairro|            endereco|           numero|detalhe_endereco_acidente|         complemento|bairro_cruzamento|num_semaforo|sentido_via|                tipo|auto|moto|ciclom|ciclista|pedestre|onibus|caminhao|viatura|outros|vitimas|vitimasfatais|acidente_verificado|tempo_clima|situacao_semaforo|sinalizacao|condicao_via|conservacao_via|ponto_controle|situacao_placa|velocidade_max_via|mao_direcao|divisao_via1|divisao_via2|divisao_via3|
+-------+-----------+------------+

### 2.2. Definição do Schema Unificado (Unified Schema Definition)

Com base na análise, definimos um **schema unificado e explícito** usando `StructType`. Este schema representa o "superconjunto" de todas as colunas encontradas em todos os arquivos, padronizando os nomes (ex: `DATA` para `data`) e definindo os tipos de dados corretos. Manter todas as colunas como `nullable=True` nesta etapa garante que a ingestão não falhe por dados faltantes na origem.

In [10]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType

unified_schema = StructType([
  StructField('data', DateType(), True),
  StructField('hora', StringType(), True),
  StructField('natureza_acidente', StringType(), True),
  StructField('situacao', StringType(), True),
  StructField('protocolo', StringType(), True),

  StructField('bairro', StringType(), True),
  StructField('endereco', StringType(), True),
  StructField('numero', StringType(), True),

  StructField('detalhe_endereco_acidente', StringType(), True),
  StructField('complemento', StringType(), True),
  StructField('endereco_cruzamento', StringType(), True),
  StructField('numero_cruzamento', StringType(), True),
  StructField('referencia_cruzamento', StringType(), True),
  StructField('bairro_cruzamento', StringType(), True),
  StructField('num_semaforo', IntegerType(), True),
  StructField('sentido_via', StringType(), True),
  StructField('tipo', StringType(), True),
  StructField('descricao', StringType(), True),

  StructField('auto', IntegerType(), True),
  StructField('moto', IntegerType(), True),
  StructField('ciclom', IntegerType(), True),
  StructField('ciclista', IntegerType(), True),
  StructField('pedestre', IntegerType(), True),
  StructField('onibus', IntegerType(), True),
  StructField('caminhao', IntegerType(), True),
  StructField('viatura', IntegerType(), True),
  StructField('outros', IntegerType(), True),
  StructField('vitimas', IntegerType(), True),
  StructField('vitimas_fatais', IntegerType(), True),

  StructField('acidente_verificado', StringType(), True),
  StructField('tempo_clima', StringType(), True),
  StructField('situacao_semaforo', StringType(), True),
  StructField('sinalizacao', StringType(), True),
  StructField('condicao_via', StringType(), True),
  StructField('conservacao_via', StringType(), True),
  StructField('ponto_controle', StringType(), True),
  StructField('situacao_placa', StringType(), True),
  StructField('velocidade_max_via', IntegerType(), True),
  StructField('mao_direcao', StringType(), True),

  StructField('divisao_via1', StringType(), True),
  StructField('divisao_via2', StringType(), True),
  StructField('divisao_via3', StringType(), True)

])

schema_columns = unified_schema.names
print(f"Your Schema have {len(schema_columns)} columns.")
print(unified_schema)

Your Schema have 42 columns.
StructType([StructField('data', DateType(), True), StructField('hora', StringType(), True), StructField('natureza_acidente', StringType(), True), StructField('situacao', StringType(), True), StructField('protocolo', StringType(), True), StructField('bairro', StringType(), True), StructField('endereco', StringType(), True), StructField('numero', StringType(), True), StructField('detalhe_endereco_acidente', StringType(), True), StructField('complemento', StringType(), True), StructField('endereco_cruzamento', StringType(), True), StructField('numero_cruzamento', StringType(), True), StructField('referencia_cruzamento', StringType(), True), StructField('bairro_cruzamento', StringType(), True), StructField('num_semaforo', IntegerType(), True), StructField('sentido_via', StringType(), True), StructField('tipo', StringType(), True), StructField('descricao', StringType(), True), StructField('auto', IntegerType(), True), StructField('moto', IntegerType(), True), St

In [11]:
import os

data_dir = 'data'
all_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')])

all_headers = set()

for file_path in all_files:
  header = spark.read.option("delimiter", ";").option("header", "true").csv(file_path).columns
  all_headers.update(header)

unique_column_list = sorted(list(all_headers))

print(f"Found {len(unique_column_list)} unique columns across all files:")
print(unique_column_list)

Found 44 unique columns across all files:
['DATA', 'Protocolo', 'acidente_verificado', 'auto', 'bairro', 'bairro_cruzamento', 'caminhao', 'ciclista', 'ciclom', 'complemento', 'condicao_via', 'conservacao_via', 'data', 'descricao', 'detalhe_endereco_acidente', 'divisao_via1', 'divisao_via2', 'divisao_via3', 'endereco', 'endereco_cruzamento', 'hora', 'mao_direcao', 'moto', 'natureza', 'natureza_acidente', 'num_semaforo', 'numero', 'numero_cruzamento', 'onibus', 'outros', 'pedestre', 'ponto_controle', 'referencia_cruzamento', 'sentido_via', 'sinalizacao', 'situacao', 'situacao_placa', 'situacao_semaforo', 'tempo_clima', 'tipo', 'velocidade_max_via', 'viatura', 'vitimas', 'vitimasfatais']


In [12]:
source_columns_standardized = [header.lower() for header in unique_column_list]
print(f"The source files have {len(source_columns_standardized)} unique standardized columns.")

extra_columns = set(schema_columns) - set(source_columns_standardized)
if extra_columns:
  print(f"The extra columns are: {extra_columns}")
else:
  print("No extra columns found.")

over_columns = set(source_columns_standardized) - set(schema_columns)
if over_columns:
  print(f"The missing columns are: {over_columns}")
else:
  print("No missing columns found.")

The source files have 44 unique standardized columns.
The extra columns are: {'vitimas_fatais'}
The missing columns are: {'natureza', 'vitimasfatais'}


### 2.3. Carga, Mapeamento e Enriquecimento (Ingestion, Mapping & Enrichment)

Implementamos um loop para processar cada arquivo CSV. Para lidar com o Schema Drift, utilizamos o padrão **"Read, Rename, and UnionByName"**:
1.  **Read:** Lemos cada arquivo individualmente, deixando o Spark inferir os nomes originais do cabeçalho.
2.  **Rename:** Renomeamos as colunas inconsistentes para se alinharem ao nosso schema padrão.
3.  **UnionByName:** Unimos o DataFrame tratado ao DataFrame final, alinhando as colunas pelo nome.

Além disso, enriquecemos os dados com duas colunas de metadados essenciais:
- `source_file`: Para rastreabilidade da origem de cada registro.
- `year`: Para permitir o particionamento da tabela.

In [13]:
df_bronze_final = spark.createDataFrame([], unified_schema)

In [14]:
data_dir = 'data'
all_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')])

for file_path in all_files:
    df_raw = spark.read.option("header", "true").option("delimiter", ";").option('inferSchema', True).csv(file_path)

    df_renamed = df_raw.withColumnRenamed('DATA','data') \
                       .withColumnRenamed('natureza', 'natureza_acidente') \
                       .withColumnRenamed('vitimasfatais', 'vitimas_fatais') \
                       .withColumnRenamed('Protocolo', 'protocolo')

    df_bronze_final = df_bronze_final.unionByName(df_renamed, allowMissingColumns=True)

In [15]:
final_df_columns = df_bronze_final.columns

expected_schema_columns = unified_schema.names

missing_columns = set(expected_schema_columns) - set(final_df_columns)
extra_columns = set(final_df_columns) - set(expected_schema_columns)

print("="*50)
print("Final Schema Validation Report")
print("="*50)

if not missing_columns and not extra_columns:
    print(f"✅ SUCCESS! The final DataFrame schema perfectly matches the expected unified schema.")
    print(f"Total columns loaded: {len(final_df_columns)}")
else:
    print("⚠️ ATTENTION: Discrepancy found!")
    if missing_columns:
        print(f"  - Columns defined in schema but MISSING from final DataFrame: {list(missing_columns)}")
    if extra_columns:
        print(f"  - EXTRA columns found in final DataFrame that were not in schema: {list(extra_columns)}")

Final Schema Validation Report
✅ SUCCESS! The final DataFrame schema perfectly matches the expected unified schema.
Total columns loaded: 42


In [16]:
from pyspark.sql.functions import input_file_name, year, col


df_bronze_with_metadata = df_bronze_final \
  .withColumn('source_file', input_file_name()) \
  .withColumn('year', year(col('data')))

print("DataFrame with source_file metadata column:")
df_bronze_with_metadata.select("data", "bairro", "source_file", "year").show(5, truncate=False)

DataFrame with source_file metadata column:
+----------+-----------+---------------------------------------+----+
|data      |bairro     |source_file                            |year|
+----------+-----------+---------------------------------------+----+
|2019-01-01|IPSEP      |file:///content/data/acidentes_2019.csv|2019|
|2019-01-01|BOA VIAGEM |file:///content/data/acidentes_2019.csv|2019|
|2019-01-01|BOA VIAGEM |file:///content/data/acidentes_2019.csv|2019|
|2019-01-01|IMBIRIBEIRA|file:///content/data/acidentes_2019.csv|2019|
|2019-01-01|JAQUEIRA   |file:///content/data/acidentes_2019.csv|2019|
+----------+-----------+---------------------------------------+----+
only showing top 5 rows



### 2.4. Salvando a Tabela Bronze (Saving the Bronze Table)

Finalmente, salvamos o DataFrame unificado e enriquecido como uma tabela Delta, que é a base do nosso Lakehouse. A tabela é salva no modo `overwrite` e **particionada por ano** (`partitionBy("year")`) para otimizar drasticamente a performance de futuras consultas que filtrem por data.*texto em itálico*

In [17]:
bronze_table_path = "delta_lake/bronze/acidentes"

print("Saving Bronze table...")

df_bronze_with_metadata.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year") \
    .save(bronze_table_path)

print(f"SUCCESS: Bronze table saved successfully to '{bronze_table_path}'")

Saving Bronze table...
SUCCESS: Bronze table saved successfully to 'delta_lake/bronze/acidentes'


## Conclusão da Camada Bronze

A Camada Bronze está completa! Nossos dados brutos de múltiplos arquivos inconsistentes estão agora armazenados em uma única tabela Delta, com schema unificado, metadados de linhagem e otimizada para leitura com particionamento.

**Próximo Passo:** Iniciar a **Camada Silver**, onde vamos limpar, validar e enriquecer estes dados.

## 3. Camada Silver: Limpeza e Enriquecimento de Dados

Iniciamos a Camada Silver, o coração do nosso processo de ETL. O objetivo aqui é transformar os dados brutos e não confiáveis da Camada Bronze em uma fonte de dados limpa, consistente e enriquecida, pronta para análises mais complexas. Aplicaremos regras de negócio e técnicas de limpeza para garantir a **Qualidade e Governança dos Dados**, como defendido por **Reis & Housley** e **Kimball**.

### 3.1. Leitura da Tabela Bronze

O primeiro passo é carregar nossa tabela Delta da Camada Bronze. Esta ação representa o início do pipeline que move os dados entre as camadas Bronze e Silver.

In [18]:
df_silver = spark.read.format("delta").load(bronze_table_path)

print("Reading from Bronze Delta table to start the Silver process...")
df_silver.printSchema()
print(f"\nTotal records read from Bronze: {df_silver.count()}")
df_silver.show(5)

Reading from Bronze Delta table to start the Silver process...
root
 |-- data: date (nullable = true)
 |-- hora: string (nullable = true)
 |-- natureza_acidente: string (nullable = true)
 |-- situacao: string (nullable = true)
 |-- protocolo: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- detalhe_endereco_acidente: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- endereco_cruzamento: string (nullable = true)
 |-- numero_cruzamento: string (nullable = true)
 |-- referencia_cruzamento: string (nullable = true)
 |-- bairro_cruzamento: string (nullable = true)
 |-- num_semaforo: string (nullable = true)
 |-- sentido_via: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- auto: string (nullable = true)
 |-- moto: string (nullable = true)
 |-- ciclom: string (nullable = true)
 |-- ciclista: string (nullable = true)


### 3.2. Limpeza e Padronização de Dados (Data Cleaning)

Esta etapa foca em melhorar a qualidade e a consistência dos dados. Seguindo as práticas de **Limpeza de Dados** de **McKinney**, aplicamos duas transformações essenciais:

* **Padronização de Case:** Convertemos colunas categóricas de texto para maiúsculas (`UPPERCASE`). Isso garante que valores como "Boa Viagem" e "BOA VIAGEM" sejam tratados como um só, um processo que **Kimball** chama de "conformar atributos de dimensão".
* **Tratamento de Nulos:** Substituímos valores `NULL` em colunas categóricas importantes (como `bairro`) por um valor explícito (`'NAO INFORMADO'`). Isso torna o dado menos ambíguo e mais fácil de ser consumido por ferramentas de BI.

In [24]:
from pyspark.sql.functions import upper, col

df_temp = df_silver \
  .withColumn('bairro', upper(col('bairro'))) \

print("Verifying the standardization of the 'bairro' column:")
df_temp.select("bairro").show(5, truncate=False)

Verifying the standardization of the 'bairro' column:
+----------+
|bairro    |
+----------+
|MADALENA  |
|BONGI     |
|JAQUEIRA  |
|BOA VIAGEM|
|BOA VIAGEM|
+----------+
only showing top 5 rows



In [26]:
columns_to_upper = ['natureza_acidente', 'situacao', 'bairro', 'endereco', 'bairro_cruzamento', 'sentido_via', 'tipo', 'tempo_clima', 'mao_direcao']

df_silver_casing = df_silver
for col_name in columns_to_upper:
    df_silver_casing = df_silver_casing.withColumn(col_name, upper(col(col_name)))

print("Verification of the loop transformation:")
df_silver_casing.select(columns_to_upper).show(5, truncate=False)

Verification of the loop transformation:
+-----------------+----------+----------+--------------------------------+-----------------+-----------+-----------------------+-----------+-----------+
|natureza_acidente|situacao  |bairro    |endereco                        |bairro_cruzamento|sentido_via|tipo                   |tempo_clima|mao_direcao|
+-----------------+----------+----------+--------------------------------+-----------------+-----------+-----------------------+-----------+-----------+
|COM VÍTIMA       |FINALIZADA|MADALENA  |AV ENGENHEIRO ABDIAS DE CARVALHO|MADALENA         |CIDADE     |ATROPELAMENTO DE PESSOA|NULL       |NULL       |
|COM VÍTIMA       |FINALIZADA|BONGI     |AV ENGENHEIRO ABDIAS DE CARVALHO|BONGI            |SUBURBIO   |COLISÃO LATERAL        |NULL       |NULL       |
|COM VÍTIMA       |FINALIZADA|JAQUEIRA  |RUA MUNIZ TAVARES               |JAQUEIRA         |CIDADE     |COLISÃO LATERAL        |NULL       |NULL       |
|SEM VÍTIMA       |FINALIZADA|BOA VIAGEM|

In [28]:
df_temp = df_silver_casing.na.fill('NÃO INFORMADO', ['bairro'])

print("\nVerification of null handling:")
df_temp.filter(col("bairro") == "NAO INFORMADO").select("data", "endereco", "bairro").show(5, truncate=False)


Verification of null handling:
+----+--------+------+
|data|endereco|bairro|
+----+--------+------+
+----+--------+------+



In [32]:
subset = ['natureza_acidente', 'situacao', 'bairro', 'sentido_via', 'tipo']

df_silver_nulls_handled = df_silver_casing.na.fill('NÃO INFORMADO', subset)

In [33]:
print("\nVerification of null handling:")
df_silver_nulls_handled.filter(col('situacao') == 'NÃO INFORMADO').select(subset).show(5, truncate=False)


Verification of null handling:
+-----------------+-------------+------------------+-------------+-------------------------+
|natureza_acidente|situacao     |bairro            |sentido_via  |tipo                     |
+-----------------+-------------+------------------+-------------+-------------------------+
|NÃO INFORMADO    |NÃO INFORMADO|BOMBA DO HEMETÉRIO|CIDADE       |CHOQUE OBJETO FIXO       |
|NÃO INFORMADO    |NÃO INFORMADO|SANTO ANTÔNIO     |SUBURBIO     |ABALROAMENTO LONGITUDINAL|
|NÃO INFORMADO    |NÃO INFORMADO|IPUTINGA          |CIDADE       |ABALROAMENTO LONGITUDINAL|
|NÃO INFORMADO    |NÃO INFORMADO|IPUTINGA          |CIDADE       |ABALROAMENTO LONGITUDINAL|
|NÃO INFORMADO    |NÃO INFORMADO|AFOGADOS          |NÃO INFORMADO|COLISÃO                  |
+-----------------+-------------+------------------+-------------+-------------------------+



### 3.3. Enriquecimento de Dados (Feature Engineering)

Nesta fase de **Transformação e Mapeamento (McKinney)**, criamos novas colunas (*features*) que agregam valor analítico e facilitam as consultas.

* **Atributos de Data:** A partir da coluna `data`, derivamos `month`, `day_of_month` e `day_of_week`. Este é um passo fundamental na criação de uma futura **Dimensão Calendário (Kimball)**.
* **Atributos de Tempo:** Usando lógica condicional (`when/otherwise`), transformamos a coluna `hora` em uma categoria `periodo_do_dia`, o que simplifica análises baseadas em períodos.

In [36]:
from pyspark.sql.functions import month, col

df_temp = df_silver_nulls_handled.withColumn('month', month(col('data')))

print("Verification of the new 'month' column:")
df_temp.select("data", "month").show(5)

Verification of the new 'month' column:
+----------+-----+
|      data|month|
+----------+-----+
|2023-01-01|    1|
|2023-01-01|    1|
|2023-01-01|    1|
|2023-01-01|    1|
|2023-01-01|    1|
+----------+-----+
only showing top 5 rows



In [42]:
from pyspark.sql.functions import month, col, date_format, day

df_silver_enriched = df_silver_nulls_handled.withColumn('day_of_month', day(col('data'))) \
  .withColumn('month', month(col('data'))) \
  .withColumn('day_of_week', date_format(col('data'), 'E'))

In [43]:
print("Verification of the new 'columns' column:")
df_silver_enriched.select("data", "month", "day_of_week", "day_of_month").show(5)

Verification of the new 'columns' column:
+----------+-----+-----------+------------+
|      data|month|day_of_week|day_of_month|
+----------+-----+-----------+------------+
|2023-01-01|    1|        Sun|           1|
|2023-01-01|    1|        Sun|           1|
|2023-01-01|    1|        Sun|           1|
|2023-01-01|    1|        Sun|           1|
|2023-01-01|    1|        Sun|           1|
+----------+-----+-----------+------------+
only showing top 5 rows



In [44]:
from pyspark.sql.functions import col, hour, when

df_silver_final = df_silver_enriched.withColumn("periodo_do_dia",
    when((hour(col("hora")) >= 6) & (hour(col("hora")) < 12), "MANHA") \
    .when((hour(col("hora")) >= 12) & (hour(col("hora")) < 18), "TARDE") \
    .when((hour(col("hora")) >= 18) & (hour(col("hora")) <= 23), "NOITE") \
    .otherwise("MADRUGADA")
)

In [45]:
print("Verification of the new 'periodo_do_dia' column:")
df_silver_final.groupBy("periodo_do_dia").count().orderBy("count", ascending=False).show()

Verification of the new 'periodo_do_dia' column:
+--------------+-----+
|periodo_do_dia|count|
+--------------+-----+
|         MANHA|11889|
|         TARDE| 8046|
|     MADRUGADA| 6565|
|         NOITE| 4050|
+--------------+-----+

